==============================\
Student name: Shiqi Yin \
GitHub username: acse-sy121\
\==============================

In [ ]:
!pip install -q vtk

     |████████████████████████████████| 88.3 MB 1.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math  import *
import datetime, time
import pprint as pprint
import vtk
import os
os.chdir("/content/drive/MyDrive")
import vtktools
import joblib

In [ ]:
root_path = ''

times = 720
nNodes = 192060

x = [10.3, 6.8, 3.1]
y = [5.2, 0.8, 5.0]
z = [0.4, 1.0, 1.5, 2.25, 3.0, 3.5]

# --------------------------#
# -- Coordinates Fluidity --#
#---------------------------#
coordinates = []

for i in range(6):
  coord1 = (x[0], y[0], z[i])
  coordinates.append(coord1)
  coord2 = (x[1], y[1], z[i])
  coordinates.append(coord2)
  coord3 = (x[2], y[2], z[i])
  coordinates.append(coord3)

coordinates = np.array(coordinates)

print ('\n Coordinates Concentration ::')
print (coordinates)


 Coordinates Concentration ::
[[10.3   5.2   0.4 ]
 [ 6.8   0.8   0.4 ]
 [ 3.1   5.    0.4 ]
 [10.3   5.2   1.  ]
 [ 6.8   0.8   1.  ]
 [ 3.1   5.    1.  ]
 [10.3   5.2   1.5 ]
 [ 6.8   0.8   1.5 ]
 [ 3.1   5.    1.5 ]
 [10.3   5.2   2.25]
 [ 6.8   0.8   2.25]
 [ 3.1   5.    2.25]
 [10.3   5.2   3.  ]
 [ 6.8   0.8   3.  ]
 [ 3.1   5.    3.  ]
 [10.3   5.2   3.5 ]
 [ 6.8   0.8   3.5 ]
 [ 3.1   5.    3.5 ]]


In [ ]:
#--------------------------------------------------#
#--  Function to initialise vtk files   --#
#--------------------------------------------------#
def Initialisation(filename):
    '''
      This function initialises the vtk file
      Parameters
      ----------
    
      filename : string
        The filename of the vtk file

      Returns
      ---------

      ugrid 
      
    '''
    # Read file
    print ('     Read file')
    if filename[-4:] == ".vtu":
        gridreader=vtk.vtkXMLUnstructuredGridReader()
    elif filename[-5:] == ".pvtu":
        gridreader=vtk.vtkXMLPUnstructuredGridReader()
    gridreader.SetFileName(filename)
    gridreader.Update()
    ugrid=gridreader.GetOutput()

    return ugrid

#-------------------------------------------------#
#-- Function to initialise probe filter  --#
#-------------------------------------------------#
def InitialisePointData(ugrid, coordinates):

    # Initialise probe
    points = vtk.vtkPoints()
    points.SetDataTypeToDouble()

    # Create points to be extracted
    print ('     Gathering  coordinates')
    NrbPoints = 0
    for nodeID in range(len(coordinates)):
        NrbPoints += 1
        points.InsertNextPoint(coordinates[nodeID][0], coordinates[nodeID][1], coordinates[nodeID][2])

    print ('           Set points into data...')
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    probe = vtk.vtkProbeFilter()

    print ('           Map data into probe...', 'VTK version ::', vtk.vtkVersion.GetVTKMajorVersion(),'.', vtk.vtkVersion.GetVTKMinorVersion())
    if vtk.vtkVersion.GetVTKMajorVersion() <= 5:
        probe.SetInput(polydata)
        probe.SetSource(ugrid)
    else:
        probe.SetInputData(polydata)
        probe.SetSourceData(ugrid)

    probe.Update()
    
    return probe, points, NrbPoints
#-------------------------------------------------#
#-- Function to initialise cell filter           -#
#-------------------------------------------------#
def InitialiseCellLocator(ugrid):

    # Initialise locator
    print ('     Initialise cell Locator')
    CellLocator = vtk.vtkCellLocator()
    CellLocator.SetDataSet(ugrid)
    CellLocator.Update()
    
    return CellLocator

In [ ]:
#--------------------------------#
#-- Choose variables           --#
#--------------------------------#
# The path where the vtu files are located
path = '/content/drive/MyDrive/Cotrace_fixed_final/' 
# The prefix of the file name of the vtu file
name_simu = 'Cotrace_fixed'
vtu_start = 1
vtu_end   = 721  # 726
vtu_step  = 1
# fields_list = ['Tracer', 'Humidity', 'Temperature', 'Virus1']
fields_list = ['Velocity']

In [ ]:
#---------------------------------------------------------------------
# EXTRACT DATA
#---------------------------------------------------------------------
def load_coord_data(filename, fields_list):
    
    #-- Fields
    FIELDS = []

    for f in range(len(fields_list)):
        tic = time.time()
        #-- CO2
        CO2     = []
        #-- Time
        TimeVTU = []
        r = 0
        CO2.append([])
        fieldname = fields_list[f]
        # print(fieldname)
        # Read file
        ugrid = Initialisation(filename)
        # Initialise probe
        probe, points, NrbPoints = InitialisePointData(ugrid, coordinates)
        # Initialise cell location
        CellLocator = InitialiseCellLocator(ugrid)
        #-- Check Validity of points
        # print ('           Check point Validity')
        valid_ids = probe.GetOutput().GetPointData().GetArray('vtkValidPointMask')
        validPoints = vtktools.arr([valid_ids.GetTuple1(i) for i in range(NrbPoints)])
        # print ('           ... ', len(validPoints)-np.sum(validPoints), 'invalid points')
        # Extract data
        # print ('     Extract Data')
        for nodeID in range(len(coordinates)):
            # If valid point, extract using probe,
            # Otherwise extract the cell:
            #    If no cell associated - then it is really a non-valid point outside the domain
            #    Otherwise: do the average over the cell values - this provide the tracer value.
            # We need to do that as it is a well-known bug in vtk libraries - sometimes it returns an invalid node while it is not...
            if validPoints[nodeID] == 1:
                tmp = probe.GetOutput().GetPointData().GetArray(fieldname).GetTuple(nodeID)
                CO2[r].append(tmp)
            else:
                coord_tmp = np.array(points.GetPoint(nodeID))
                cellID =  CellLocator.FindCell(coord_tmp) # cell ID which contains the node
                idlist=vtk.vtkIdList()
                ugrid.GetCellPoints(cellID, idlist)
                pointsID_to_cellID = np.array([idlist.GetId(k) for k in range(idlist.GetNumberOfIds())]) # give all the points asociated with this cell
                if len(pointsID_to_cellID) == 0: # Non-valid points - We assign negative value - like that we know we are outside the domain
                    CO2[r].append(-1e20)
                else:
                    tmp = 0
                    for pointID in pointsID_to_cellID:
                        tmp += ugrid.GetPointData().GetArray(fieldname).GetTuple(pointID)[0]
                    tmp = tmp/len(pointsID_to_cellID)
                    CO2[r].append(tmp)
            # print("nodeID: ", valid_ids)

        # Time
        time_tmp = probe.GetOutput().GetPointData().GetArray('Time').GetValue(0)
        TimeVTU.append(time_tmp)
        # print ('      Time ::', time_tmp)
        # print ('      CO2  ::', CO2[r])

        r += 1

        # print ('\n Time in (s) ::')
        # print (TimeVTU)    
        # np.savetxt('TimeVTU.dat',TimeVTU)

        print ('\n CO2 ::')
        CO2 = np.array(CO2)
        # print (CO2)
        CO2_all = [np.array(TimeVTU)]
        # CO2_all = np.transpose(np.append(CO2_all, np.transpose(CO2), axis = 0))

        # joblib.dump(CO2, '/content/gdrive/MyDrive/ACSE-9/models/sensor_CO2_410.pkl') #CO2 data -> ( nSnapshot x 7(sensors) )
        # joblib.dump(CO2_all, '/content/gdrive/MyDrive/ACSE-9/models/CO2_Fluidity_410.pkl') #with the time column
        # joblib.dump(CO2, '/content/gdrive/MyDrive/ACSE-9/models/sensor_CO2_all.pkl') #CO2 data -> ALL time levels

        toc = time.time()

        # print ('\n\nTime : ', toc - tic, 'sec')

        FIELDS.append(CO2)

    FIELDS = np.array(FIELDS)

    return FIELDS


In [ ]:
vtu_start = 1
vtu_end   = 720  # 726
vtu_step  = 1


results = []
for t in range(vtu_start, vtu_end+1, vtu_step):
    filename = path + 'Cotrace_fixed_{}'.format(t) + '.vtu'
    print(filename)
    result = load_coord_data(filename, fields_list)
    results.append(result)
results_array = np.array(results)


# results_array = results_array.reshape(120, 5, 18)

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/Cotrace_fixed_final/Cotrace_fixed_96.vtu
     Read file
     Gathering  coordinates
           Set points into data...
           Map data into probe... VTK version :: 9 . 1
     Initialise cell Locator

 CO2 ::
/content/drive/MyDrive/Cotrace_fixed_final/Cotrace_fixed_97.vtu
     Read file
     Gathering  coordinates
           Set points into data...
           Map data into probe... VTK version :: 9 . 1
     Initialise cell Locator

 CO2 ::
/content/drive/MyDrive/Cotrace_fixed_final/Cotrace_fixed_98.vtu
     Read file
     Gathering  coordinates
           Set points into data...
           Map data into probe... VTK version :: 9 . 1
     Initialise cell Locator

 CO2 ::
/content/drive/MyDrive/Cotrace_fixed_final/Cotrace_fixed_99.vtu
     Read file
     Gathering  coordinates
           Set points into data...
           Map data into probe... VTK version :: 9 . 1
     Initialise cell Locator

 CO2 ::
/content/drive/MyDrive/Cotrace_fi

In [ ]:
print(results_array.shape)

(720, 1, 1, 18, 3)


In [ ]:
results_array = results_array.reshape(720, 3, 18)

In [ ]:
results_array.shape

(720, 3, 18)

Other fields

In [ ]:
# np.save('/content/drive/MyDrive/observation.npy',results_array)

Velocity field

In [ ]:
np.save('/content/drive/MyDrive/observation1.npy',results_array)